<a href="https://colab.research.google.com/github/sayakpaul/Image-Adversaries-101/blob/master/Optimizer_Susceptibility_Targeted_Attacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Contents of this notebook are referred from here: https://adversarial-ml-tutorial.org/introduction/.

## Setup

In [ ]:
!pip install wandb -qq

In [ ]:
import wandb
wandb.login()

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import decode_predictions
import matplotlib.pyplot as plt
import numpy as np
import cv2

## Utils

In [ ]:
def show_image(image_path):
    image_pixels = plt.imread(image_path)
    plt.imshow(image_pixels)
    plt.show()
    return image_pixels

In [ ]:
def preprocess_image(image_pixels):
    image_pixels = preprocess_input(image_pixels)
    image_pixels = cv2.resize(image_pixels, (224, 224))
    image_pixels = np.expand_dims(image_pixels, axis=0)

    return image_pixels

## Pretrained network and an image to play with

In [ ]:
# Load ResNet50
resnet50 = tf.keras.applications.ResNet50(weights="imagenet")

In [ ]:
!wget https://adversarial-ml-tutorial.org/introduction/pig.jpg

## Pretrained network and an image to play with

In [ ]:
# Preprocess the image
sample_image = show_image("pig.jpg")
preprocessed_image = preprocess_image(sample_image)

# Run inference
original_preds = resnet50.predict(preprocessed_image)
preds_decoded = decode_predictions(original_preds, top=3)[0]
print('Predicted:', preds_decoded)

## Utils for generating a perturbed image for tageted attacks

In [ ]:
def preprocess_image_without(image_pixels):
    image_pixels = cv2.resize(image_pixels, (224, 224))
    image_pixels = np.expand_dims(image_pixels, axis=0)

    return image_pixels

In [ ]:
# Clipping utility so that the pixel values stay within [0,1]
EPS = 2./255

def clip_eps(delta_tensor):
    return tf.clip_by_value(delta_tensor, clip_value_min=-EPS, clip_value_max=EPS)

In [ ]:
def generate_adversary(iterations, source_image, delta, 
                       true_class_idx, target_index, optimizer):
    scc_loss = tf.keras.losses.SparseCategoricalCrossentropy()

    for t in range(iterations):
        with tf.GradientTape() as tape:
            tape.watch(delta)
            inp = preprocess_input(source_image + delta)
            predictions = resnet50(inp, training=False)
            loss = (- scc_loss(tf.convert_to_tensor([true_class_idx]), predictions) + 
	                    scc_loss(tf.convert_to_tensor([target_index]), predictions))
        if t % 20 == 0:
            print(t, loss.numpy())
    
        # Get the gradients
        gradients = tape.gradient(loss, delta)
        
        # Update the weights
        optimizer.apply_gradients([(gradients, delta)])

        # Clip so that the delta values are within [0,1]
        delta.assign_add(clip_eps(delta))

        # Log to wandb
        wandb.log({'loss': loss.numpy()})
        wandb.log({'delta': wandb.Image(50*delta.numpy().squeeze()+0.5)})
        wandb.log({'perturbed_image': 
                   wandb.Image((source_image + delta).numpy().squeeze()/255)})

    return delta

In [ ]:
def log_final_results_wandb(sample_image, original_predicted_label, perturbed_image,
                            adv_predicted_label, delta):
    plt.figure(figsize=(14, 14))
    plt.subplot(1, 3, 1)
    plt.imshow(sample_image)
    plt.title(f'Original Image predicted as: \n{original_predicted_label}')
    plt.subplot(1, 3, 2)
    plt.imshow(perturbed_image)
    plt.title(f'Perturbed Image predicted as: \n{adv_predicted_label}')
    plt.subplot(1, 3, 3)
    plt.imshow(delta)
    plt.title('delta')
    plt.savefig('results.png', bbox_inches='tight', pad_inches=0)
    wandb.log({'final_results': wandb.Image('results.png')})

In [ ]:
def prepare_image(img_path):
    # Load and preprocess image but a but without any `preprocess_input`
    preprocessed_image = preprocess_image_without(plt.imread(img_path))

    # Convert the source image to a TensorFlow tensor and initialize
    # delta
    image_tensor = tf.constant(preprocessed_image, dtype=tf.float32)
    delta = tf.Variable(tf.zeros_like(image_tensor), trainable=True)

    return image_tensor, delta

## Targeted attacks

### Adam

In [ ]:
# Generate a perturbed image
wandb.init(entity='authors', project='adv-dl', id='targeted-attack-adam-I')
image_tensor, delta = prepare_image('pig.jpg')
delta_optimized = generate_adversary(350, image_tensor, delta, 
    341, 189,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1))

In [ ]:
# Parse the predictions
original_predicted_label = decode_predictions(original_preds, top=3)[0][0][1]
adv_preds = resnet50.predict(preprocess_input(image_tensor + delta_optimized))
adv_predicted_label = decode_predictions(adv_preds, top=3)[0][0][1]

print("Original image predicted as ",original_predicted_label)
print("Perturbed image predicted as ",adv_predicted_label)

log_final_results_wandb(sample_image, 
    original_predicted_label, 
    (image_tensor + delta_optimized).numpy().squeeze()/255,
    adv_predicted_label, 
    50*delta_optimized.numpy().squeeze()+0.5
)

### SGD

In [ ]:
# Generate a perturbed image
wandb.init(entity='authors', project='adv-dl', id='targeted-attack-sgd-I')
image_tensor, delta = prepare_image('pig.jpg')
delta_optimized = generate_adversary(350, image_tensor, delta, 
    341, 189,
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1))

In [ ]:
# Parse the predictions
original_predicted_label = decode_predictions(original_preds, top=3)[0][0][1]
adv_preds = resnet50.predict(preprocess_input(image_tensor + delta_optimized))
adv_predicted_label = decode_predictions(adv_preds, top=3)[0][0][1]

print("Original image predicted as ",original_predicted_label)
print("Perturbed image predicted as ",adv_predicted_label)

log_final_results_wandb(sample_image, 
    original_predicted_label, 
    (image_tensor + delta_optimized).numpy().squeeze()/255,
    adv_predicted_label, 
    50*delta_optimized.numpy().squeeze()+0.5
)

### RMSProp

In [ ]:
# Generate a perturbed image
wandb.init(entity='authors', project='adv-dl', id='targeted-attack-rmsprop-I')
image_tensor, delta = prepare_image('pig.jpg')
delta_optimized = generate_adversary(350, image_tensor, delta, 
    341, 189,
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.1))

In [ ]:
# Parse the predictions
original_predicted_label = decode_predictions(original_preds, top=3)[0][0][1]
adv_preds = resnet50.predict(preprocess_input(image_tensor + delta_optimized))
adv_predicted_label = decode_predictions(adv_preds, top=3)[0][0][1]

print("Original image predicted as ",original_predicted_label)
print("Perturbed image predicted as ",adv_predicted_label)

log_final_results_wandb(sample_image, 
    original_predicted_label, 
    (image_tensor + delta_optimized).numpy().squeeze()/255,
    adv_predicted_label, 
    50*delta_optimized.numpy().squeeze()+0.5
)

### Adagrad

In [ ]:
# Generate a perturbed image
wandb.init(entity='authors', project='adv-dl', id='targeted-attack-adagrad-I')
image_tensor, delta = prepare_image('pig.jpg')
delta_optimized = generate_adversary(350, image_tensor, delta, 
    341, 189,
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
# Parse the predictions
original_predicted_label = decode_predictions(original_preds, top=3)[0][0][1]
adv_preds = resnet50.predict(preprocess_input(image_tensor + delta_optimized))
adv_predicted_label = decode_predictions(adv_preds, top=3)[0][0][1]

print("Original image predicted as ",original_predicted_label)
print("Perturbed image predicted as ",adv_predicted_label)

log_final_results_wandb(sample_image, 
    original_predicted_label, 
    (image_tensor + delta_optimized).numpy().squeeze()/255,
    adv_predicted_label, 
    50*delta_optimized.numpy().squeeze()+0.5
)

### FTRL

In [ ]:
# Generate a perturbed image
wandb.init(entity='authors', project='adv-dl', id='targeted-attack-ftrl-I')
image_tensor, delta = prepare_image('pig.jpg')
delta_optimized = generate_adversary(350, image_tensor, delta, 
    341, 189,
    optimizer=tf.keras.optimizers.Ftrl(learning_rate=0.1))

In [ ]:
# Parse the predictions
original_predicted_label = decode_predictions(original_preds, top=3)[0][0][1]
adv_preds = resnet50.predict(preprocess_input(image_tensor + delta_optimized))
adv_predicted_label = decode_predictions(adv_preds, top=3)[0][0][1]

print("Original image predicted as ",original_predicted_label)
print("Perturbed image predicted as ",adv_predicted_label)

log_final_results_wandb(sample_image, 
    original_predicted_label, 
    (image_tensor + delta_optimized).numpy().squeeze()/255,
    adv_predicted_label, 
    50*delta_optimized.numpy().squeeze()+0.5
)